In [ ]:
import os
import soundfile as sf
import librosa
import numpy as np
from scipy import signal
from IPython.display import Audio

In [ ]:
audio_path = "/media/jakki/Seagate_Expansion_Drive/keparoi/2023/matsi_13062023/tmp/m04qs9i4.wav"

In [ ]:
y1, sr1 = librosa.load(audio_path)

In [ ]:
start = 60 * 10 * sr1
end = 60 * 11 * sr1

y1 = y1[start:end]

In [ ]:
def butter_lowpass_filter(data: np.ndarray, cutoff: float, samplerate: float, order: int = 5) -> np.ndarray:
    nyq = 0.5 * samplerate
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    y = signal.filtfilt(b, a, data)
    return y


def butter_highpass_filter(data: np.ndarray, cutoff: float, samplerate: float, order: int = 5) -> np.ndarray:
    nyq = 0.5 * samplerate
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

def butter_bandpass_filter(data: np.ndarray, lowcut: float, highcut: float, samplerate: float, order: int = 5) -> np.ndarray:
    nyq = 0.5 * samplerate
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    y = signal.filtfilt(b, a, data)
    return y

def spectral_noise_reduction(audio: np.ndarray, sr: int) -> np.ndarray:
    """Remove noise using spectral gating."""
    D = librosa.stft(audio)
    
    noise_profile = np.mean(np.abs(D[:, :sr//2]), axis=1)
    
    D_denoised = D * (np.abs(D) > 2 * noise_profile[:, np.newaxis])
    
    return librosa.istft(D_denoised)

def remove_wind_noise(audio: np.ndarray, frame_size: int = 2048, overlap: float = 0.75) -> np.ndarray:
    """Remove wind noise using spectral subtraction.
    
    Args:
        audio: Input audio
        frame_size: Size of FFT frames (default 2048)
        overlap: Overlap between frames (default 0.75)
    
    Returns:
        Processed audio with reduced wind noise
    """
    hop_length = int(frame_size * (1 - overlap))
    frames = librosa.util.frame(audio, frame_length=frame_size, hop_length=hop_length)
    window = np.hanning(frame_size)
    
    processed_frames = []
    for frame in frames.T:
        windowed = frame * window
        fft = np.fft.rfft(windowed)
        magnitude = np.abs(fft)
        phase = np.angle(fft)
        
        # Focus on low frequency range where wind noise typically occurs (0-200 Hz)
        wind_region = magnitude[:int(len(magnitude) * 0.05)]  # First 5% of frequencies
        noise_floor = np.mean(wind_region)
        
        # More conservative gain calculation
        gain = np.ones_like(magnitude)
        
        # Only apply reduction to low frequencies
        low_freq_idx = int(len(magnitude) * 0.1)  # First 10% of frequencies
        gain[:low_freq_idx] = np.maximum(0.3, 1 - (noise_floor / (magnitude[:low_freq_idx] + 1e-10)))
        
        # Smooth transitions
        gain = signal.medfilt(gain, 3)
        
        # Apply gain and reconstruct
        magnitude_clean = magnitude * gain
        fft_clean = magnitude_clean * np.exp(1j * phase)
        frame_clean = np.fft.irfft(fft_clean)
        
        processed_frames.append(frame_clean[:frame_size])
    
    # Overlap-add reconstruction
    output = np.zeros(len(audio))
    for i, frame in enumerate(processed_frames):
        start = i * hop_length
        end = start + frame_size
        if end > len(output):
            break
        output[start:end] += frame
    
    # Normalize but preserve some headroom
    output = 0.95 * (output / np.max(np.abs(output)))
    
    return output


In [ ]:
Audio(y1, rate=sr1)

In [ ]:
y1_lowpass = butter_lowpass_filter(y1, cutoff=8000, samplerate=sr1)

In [ ]:
Audio(y1_lowpass, rate=sr1)

In [ ]:
y1_highpass = butter_highpass_filter(y1, cutoff=300, samplerate=sr1)

In [ ]:
Audio(y1_highpass, rate=sr1)

In [ ]:
y1_bandpass = butter_bandpass_filter(y1, lowcut=300, highcut=8000, samplerate=sr1)

In [ ]:
Audio(y1_bandpass, rate=sr1)

In [ ]:
y1_filtered = librosa.effects.preemphasis(y1_bandpass)

In [ ]:
Audio(y1_filtered, rate=sr1)

In [ ]:
y1_spectral_noise = spectral_noise_reduction(y1, sr1)

In [ ]:
Audio(y1_spectral_noise, rate=sr1)

In [ ]:
y1_windoff = remove_wind_noise(y1)

In [ ]:
Audio(y1_windoff, rate=sr1)

In [ ]:
y1_windoff_band = remove_wind_noise(y1_bandpass)

In [ ]:
Audio(y1_windoff_band, rate=sr1)

In [ ]:
audio2 = "/media/jakki/Seagate_Expansion_Drive/keparoi/2024/matsi_08062024/right/GOPR0044.WAV"

In [ ]:
y2, sr2 = librosa.load(audio2)

In [ ]:
start = 0
end = 60 * 2 * sr2

y2 = y2[start:end]

In [ ]:
y2_bandpass = butter_bandpass_filter(y2, lowcut=6500, highcut=8500, samplerate=sr2)

In [ ]:
Audio(y2_bandpass, rate=sr2)